In [10]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    missing_keys = valid_keys - data_keys
    extra_keys = data_keys - valid_keys

    if missing_keys or extra_keys:
        missing_message = ('missing keys: ' + ', '.join({f'{path}.{key}' for key in missing_keys})) \
                            if missing_keys else ''
        extra_message = ('extra keys: ' + ', '.join(f'{path}.{key}' for key in extra_keys)) \
                            if extra_keys else ''
        return False, ' '.join((missing_message, extra_message))
    else:
        return True, None


print(match_keys({'a': 1, 'b': 2, 'c': 3}, {'a': 1}, None))


(False, ' extra keys: None.c, None.b')
